<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Self_Supervised_Learning_(SSL).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

class ContrastiveModel(nn.Module):
    def __init__(self, base_model):
        super(ContrastiveModel, self).__init__()
        in_features = base_model.fc.in_features
        self.encoder = base_model
        self.encoder.fc = nn.Identity()  # Remove the classifier layer
        self.projection = nn.Linear(in_features, 128)

    def forward(self, x):
        x = self.encoder(x)
        x = self.projection(x)
        return F.normalize(x, dim=-1)

def contrastive_loss(features1, features2, temperature=0.5):
    batch_size = features1.size(0)
    labels = torch.cat([torch.arange(batch_size), torch.arange(batch_size)])
    features = torch.cat([features1, features2], dim=0)
    similarity_matrix = F.cosine_similarity(features.unsqueeze(1), features.unsqueeze(0), dim=-1)
    logits = similarity_matrix / temperature
    loss = F.cross_entropy(logits, labels)
    return loss

# Example usage
base_model = models.resnet18(weights=None)
model = ContrastiveModel(base_model)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(10):
    img1, img2 = torch.randn(32, 3, 224, 224), torch.randn(32, 3, 224, 224)  # Simulated image pairs
    features1 = model(img1)
    features2 = model(img2)
    loss = contrastive_loss(features1, features2)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item():.4f}')